In [1]:
# Load the JuMP related packages and several solvers
# ECOS - for solving SOCP problems
# Juniper & Ipopt - for solving MI-NLP problems
using JuMP, Juniper, ECOS, Ipopt
# Load the data/file processing related packages
using CSV, Glob, DataFrames, Statistics
# Load the Plot package for illustrating the solution
using Plots
# Load the custom functions for benchmarking  
include("./func.jl");


In [2]:
# load the full data set!
files = glob( "*_train.csv", "./ftec_project_files/");
dfs = DataFrame.( CSV.File.( files ) );
T = 800; n = length(dfs);
stocks_retur_full = zeros(T,n);
for i = 1:n
    # compute the realized return R_i(t)
    stocks_retur_full[:,i] = (dfs[i].close-dfs[i].open) ./ dfs[i].open;
end
names_stocks_full = [ dfs[i].Name[1] for i in 1:n ];
# calculate r_i and Sigma
bar_R_full = [ mean( stocks_retur_full[:,i] ) for i in 1:length(dfs) ];
Sigma_full = [ mean( (stocks_retur_full[:,i].-bar_R_full[i]).*(stocks_retur_full[:,j].-bar_R_full[j]) ) for i=1:n, j=1:n ]; 

In [3]:
# your code/functions here
B = 20;
Rd = 1.01*sum(bar_R_full);
function calc_gamma(delta)
    delta = 0.01;
    L = a/delta + max(diag(Sigma_full));
    gamma = 1/L;
    return gamma;
end

function gd(x,w,a,lambda,upsilon,delta)
    @assert length(x)==n;
    grad = Sigma_full*x+Sigma_full*w - lambda.*bar_R_full+upsilon.*ones(n);
    for i=1:n
        if abs(x[i])> delta
            grad[i] += sign(x[i])*upsilon*a;
        else
            grad[i] += upsilon*(a/delta)*x[i];
        end
    end
    return grad;
end

function obj_v(x,w,a,lambda,upsilon,delta)
    @assert length(x)==n;
    obj = 1/2*((x+w)'*Sigma_full*(x+w))+lambda*(Rd-bar_R_full'*x-bar_R_full'*w)+(upsilon*sum(x)-upsilon*B);
    for i=1:n
        if abs(x[i])>delta
            obj += upsilon*(a*abs(x[i])-a*delta/2);
        else
            obj += upsilon*(a/(2*delta)*x[i]^2);
        end
    end
    return obj;
end
            

obj_v (generic function with 1 method)

In [ ]:
# set the parameters as specified by the problem
M = 20; w = 1*ones(n); a = 1; upsilon = 10;delta = 0.01;

# calculate l and u - your code here (should be a simple formula)
u = M.*ones(n);
l = -w;
outfile = "output.txt";
score = []
# initialize the algorithm
for lambda = 30000:5:50000
    x_custom = zeros(n); 
    store_obj = []
    push!(store_obj, obj_v(x_custom,w,a,lambda,upsilon,delta) ) # replace ".." with the function you wrote for computing the objective val.

    for iteration_no = 1 : 50000 # feel free to adjust the number of iterations run here.
        # your code here
        gamma = 1/(iteration_no+1);
        x_custom -= gamma* gd(x_custom,w,a,lambda,upsilon,delta);
        for i=1:n
            if x_custom[i] > u[i]
                x_custom[i] = u[i];
            elseif x_custom[i] < l[i]
                x_custom[i] = l[i];
            end
        end
        push!(store_obj, obj_v(x_custom,w,a,lambda,upsilon,delta) ) # replace ".." with the function you wrote for computing the objective val.
    end
    x_custom_pp = copy( x_custom )
    x_custom_pp[ abs.(x_custom_pp) .< delta ] .= 0;
    sharpe_PPGD = sharpe_ratio( "./ftec_project_files/", x_custom_pp, ones(n), 2 )
    push!(score,sharpe_PPGD[1]/0.1+sharpe_PPGD[2]/1.5+200/sharpe_PPGD[3]);
    end
end
open(outfile, "w") do f
    for s in score
        println(f, score);
    end
end

Sharpe Ratio = 0.09058535999059683, Return = 1.001242548149325, Tx Cost = 350, Portfo Value = 2296.6671808641436Sharpe Ratio = 0.09059093069340605, Return = 1.00162374567756, Tx Cost = 352, Portfo Value = 2297.4745540328354Sharpe Ratio = 0.09059966681714904, Return = 1.0020305080150478, Tx Cost = 352, Portfo Value = 2298.2663952374824Sharpe Ratio = 0.09060937484416598, Return = 1.002443878342787, Tx Cost = 352, Portfo Value = 2299.049567620816Sharpe Ratio = 0.09061907541606164, Return = 1.0028574359938307, Tx Cost = 352, Portfo Value = 2299.833176514592Sharpe Ratio = 0.09062875341847179, Return = 1.0032691264299993, Tx Cost = 352, Portfo Value = 2300.611338250156Sharpe Ratio = 0.09063835215409327, Return = 1.0036706461377332, Tx Cost = 352, Portfo Value = 2301.3579649022836Sharpe Ratio = 0.09064794518139116, Return = 1.0040720122595266, Tx Cost = 352, Portfo Value = 2302.1041467689033Sharpe Ratio = 0.09065753400241941, Return = 1.0044739240090919, Tx Cost = 352, Portfo Value = 2302.851

Sharpe Ratio = 0.09126504035344801, Return = 1.0317541230760858, Tx Cost = 358, Portfo Value = 2352.261559528867Sharpe Ratio = 0.09127346245867354, Return = 1.0321495897966375, Tx Cost = 358, Portfo Value = 2352.9620758276033Sharpe Ratio = 0.09128187517386942, Return = 1.0325447770344751, Tx Cost = 358, Portfo Value = 2353.662126810321Sharpe Ratio = 0.0912902772140147, Return = 1.0329393928839048, Tx Cost = 358, Portfo Value = 2354.361021716669Sharpe Ratio = 0.09129867292034832, Return = 1.033334271074108, Tx Cost = 358, Portfo Value = 2355.0604671544393Sharpe Ratio = 0.09130706373614396, Return = 1.033729297664937, Tx Cost = 358, Portfo Value = 2355.7602085962935Sharpe Ratio = 0.09131500778493094, Return = 1.0341116699179553, Tx Cost = 358, Portfo Value = 2356.444180456441Sharpe Ratio = 0.09132235962264965, Return = 1.0344771256450496, Tx Cost = 358, Portfo Value = 2357.107084193094Sharpe Ratio = 0.09132970749359097, Return = 1.0348425611649688, Tx Cost = 358, Portfo Value = 2357.7698

Sharpe Ratio = 0.0916646621930344, Return = 1.055838410983981, Tx Cost = 366, Portfo Value = 2399.833560659138Sharpe Ratio = 0.09166885918351383, Return = 1.056143403304714, Tx Cost = 366, Portfo Value = 2400.482665720777Sharpe Ratio = 0.09167305123920076, Return = 1.0564491385396058, Tx Cost = 366, Portfo Value = 2401.1334568202383Sharpe Ratio = 0.09167723899897617, Return = 1.0567542775891468, Tx Cost = 366, Portfo Value = 2401.7829467830197Sharpe Ratio = 0.09168142211162303, Return = 1.0570594047854638, Tx Cost = 366, Portfo Value = 2402.4324014888416Sharpe Ratio = 0.09168559875636677, Return = 1.0573650553183391, Tx Cost = 366, Portfo Value = 2403.0831186755586Sharpe Ratio = 0.09168977107356363, Return = 1.0576706741613073, Tx Cost = 366, Portfo Value = 2403.7337989987823Sharpe Ratio = 0.0916939430505584, Return = 1.0579758416276444, Tx Cost = 366, Portfo Value = 2404.3832944116784Sharpe Ratio = 0.09169810860026589, Return = 1.0582811562203533, Tx Cost = 366, Portfo Value = 2405.03

Sharpe Ratio = 0.09205183488129333, Return = 1.08065323622264, Tx Cost = 382, Portfo Value = 2452.06685607307Sharpe Ratio = 0.09205821235065761, Return = 1.0810562938344102, Tx Cost = 382, Portfo Value = 2452.9072243516466Sharpe Ratio = 0.09206458589345586, Return = 1.0814587540714637, Tx Cost = 382, Portfo Value = 2453.746145325963Sharpe Ratio = 0.09207095161943421, Return = 1.081861201346509, Tx Cost = 382, Portfo Value = 2454.5850250361277Sharpe Ratio = 0.0920773097303243, Return = 1.0822638441203098, Tx Cost = 382, Portfo Value = 2455.424350100243Sharpe Ratio = 0.09208366186518428, Return = 1.0826670030263161, Tx Cost = 382, Portfo Value = 2456.2648625601696Sharpe Ratio = 0.09209000832254204, Return = 1.0830693315665643, Tx Cost = 382, Portfo Value = 2457.1034106883644Sharpe Ratio = 0.09209634874190169, Return = 1.0834717164281897, Tx Cost = 382, Portfo Value = 2457.9420715693086Sharpe Ratio = 0.09210268102244899, Return = 1.0838746479125676, Tx Cost = 382, Portfo Value = 2458.7820

Sharpe Ratio = 0.09248984220559864, Return = 1.1105969311315607, Tx Cost = 384, Portfo Value = 2514.1543150107345Sharpe Ratio = 0.09249503742041391, Return = 1.1109869529572125, Tx Cost = 384, Portfo Value = 2514.9555054438542Sharpe Ratio = 0.09250022630362785, Return = 1.1113765162075477, Tx Cost = 384, Portfo Value = 2515.7556918672176Sharpe Ratio = 0.0925054097902417, Return = 1.111766492972426, Tx Cost = 384, Portfo Value = 2516.556834542171Sharpe Ratio = 0.09251058855585478, Return = 1.112155414878232, Tx Cost = 384, Portfo Value = 2517.35565311855Sharpe Ratio = 0.09251576153631486, Return = 1.1125454668714279, Tx Cost = 384, Portfo Value = 2518.15692352694Sharpe Ratio = 0.09252093070383041, Return = 1.1129357460187925, Tx Cost = 384, Portfo Value = 2518.9586574236Sharpe Ratio = 0.09252609368671645, Return = 1.1133252545498793, Tx Cost = 384, Portfo Value = 2519.7587574043755Sharpe Ratio = 0.09253125070851108, Return = 1.113715237203072, Tx Cost = 384, Portfo Value = 2520.55987778

...

Sharpe Ratio = 0.08594450143696998, Return = 1.406335309734869, Tx Cost = 482, Portfo Value = 3526.958838265945Sharpe Ratio = 0.08593555537790229, Return = 1.4064589301911565, Tx Cost = 482, Portfo Value = 3527.6538720566296Sharpe Ratio = 0.08592660261516652, Return = 1.406582656982555, Tx Cost = 482, Portfo Value = 3528.3495340508725Sharpe Ratio = 0.08591766096625914, Return = 1.4067063132933877, Tx Cost = 482, Portfo Value = 3529.0447054350616Sharpe Ratio = 0.08590871494529297, Return = 1.4068300090148342, Tx Cost = 482, Portfo Value = 3529.7402404234344Sharpe Ratio = 0.08589978745448625, Return = 1.4069535496278596, Tx Cost = 482, Portfo Value = 3530.4347921219746Sharpe Ratio = 0.08589119806959664, Return = 1.4070868887569488, Tx Cost = 484, Portfo Value = 3531.1405692024227Sharpe Ratio = 0.08588541411669848, Return = 1.4072608796038881, Tx Cost = 484, Portfo Value = 3531.83167496028Sharpe Ratio = 0.08587961022560547, Return = 1.4074352216055657, Tx Cost = 484, Portfo Value = 3532.5

106296568294Sharpe Ratio = 0.08540184822713622, Return = 1.4180878798307968, Tx Cost = 492, Portfo Value = 3579.846721682323Sharpe Ratio = 0.08539480459907613, Return = 1.4182580164926042, Tx Cost = 492, Portfo Value = 3580.585616367324Sharpe Ratio = 0.08538775808105441, Return = 1.4184281749478782, Tx Cost = 492, Portfo Value = 3581.324780412282Sharpe Ratio = 0.08538069841617846, Return = 1.41859863090752, Tx Cost = 492, Portfo Value = 3582.0652567731604Sharpe Ratio = 0.08537364647650014, Return = 1.4187690091270502, Tx Cost = 492, Portfo Value = 3582.8053450993457Sharpe Ratio = 0.08536660020844927, Return = 1.4189393296181303, Tx Cost = 492, Portfo Value = 3583.5451457396953Sharpe Ratio = 0.08535956044187736, Return = 1.4191078707283191, Tx Cost = 492, Portfo Value = 3584.28006522163Sharpe Ratio = 0.08535256886774623, Return = 1.4192682173052862, Tx Cost = 492, Portfo Value = 3584.9913951994067Sharpe Ratio = 0.0853456027673755, Return = 1.419428131278189, Tx Cost = 492, Portfo Value 

Sharpe Ratio = 0.08488333360566855, Return = 1.4299830587584992, Tx Cost = 500, Portfo Value = 3633.626064172385Sharpe Ratio = 0.08487638064793544, Return = 1.4301533208143307, Tx Cost = 500, Portfo Value = 3634.3838520392815Sharpe Ratio = 0.08486942801427563, Return = 1.4303236843585, Tx Cost = 500, Portfo Value = 3635.141980416259Sharpe Ratio = 0.08486247764680926, Return = 1.4304939860383137, Tx Cost = 500, Portfo Value = 3635.8999383195296Sharpe Ratio = 0.0848555321765985, Return = 1.430664323487888, Tx Cost = 500, Portfo Value = 3636.657859675098Sharpe Ratio = 0.08484857899586619, Return = 1.4308347885328185, Tx Cost = 500, Portfo Value = 3637.416506520518Sharpe Ratio = 0.08484163684970852, Return = 1.4310050686480411, Tx Cost = 500, Portfo Value = 3638.174332304615Sharpe Ratio = 0.08483470127579533, Return = 1.4311752685131078, Tx Cost = 500, Portfo Value = 3638.9317511445033Sharpe Ratio = 0.08482775831155777, Return = 1.43134565137163, Tx Cost = 500, Portfo Value = 3639.69002315

6871967094967Sharpe Ratio = 0.08444182591694441, Return = 1.4438250495351481, Tx Cost = 506, Portfo Value = 3690.467325515378Sharpe Ratio = 0.08443665686811434, Return = 1.4440261106147054, Tx Cost = 506, Portfo Value = 3691.245971226618Sharpe Ratio = 0.08443148518222536, Return = 1.4442272273976953, Tx Cost = 506, Portfo Value = 3692.0248804362864Sharpe Ratio = 0.08442631742671985, Return = 1.4444282914029964, Tx Cost = 506, Portfo Value = 3692.8035951742286Sharpe Ratio = 0.0844211365341298, Return = 1.4446297016925704, Tx Cost = 506, Portfo Value = 3693.5837616694753Sharpe Ratio = 0.08441597128547708, Return = 1.444830769403415, Tx Cost = 506, Portfo Value = 3694.362465966956Sharpe Ratio = 0.08441080561007033, Return = 1.4450318538969444, Tx Cost = 506, Portfo Value = 3695.1412764944853Sharpe Ratio = 0.08440565300849803, Return = 1.4452326438780136, Tx Cost = 506, Portfo Value = 3695.9188418024196Sharpe Ratio = 0.08440048405413694, Return = 1.4454338678141847, Tx Cost = 506, Portfo V

Sharpe Ratio = 0.08412706132994714, Return = 1.4582270828856483, Tx Cost = 506, Portfo Value = 3743.804613968191Sharpe Ratio = 0.08412465855688461, Return = 1.4584414374472066, Tx Cost = 506, Portfo Value = 3744.49963809158Sharpe Ratio = 0.08412225720158077, Return = 1.4586557132134061, Tx Cost = 506, Portfo Value = 3745.194385869495Sharpe Ratio = 0.08411986037489291, Return = 1.4588696300283284, Tx Cost = 506, Portfo Value = 3745.8880143642455Sharpe Ratio = 0.08411745631414512, Return = 1.459083984391705, Tx Cost = 506, Portfo Value = 3746.5831483268976Sharpe Ratio = 0.08411505646693487, Return = 1.4592980447116917, Tx Cost = 506, Portfo Value = 3747.2773227814023Sharpe Ratio = 0.08411265554644533, Return = 1.4595122992091014, Tx Cost = 506, Portfo Value = 3747.972032445634Sharpe Ratio = 0.08411025403265007, Return = 1.4597265730009836, Tx Cost = 506, Portfo Value = 3748.6668500880783Sharpe Ratio = 0.08410785408954563, Return = 1.4599407727323965, Tx Cost = 506, Portfo Value = 3749.36